<a href="https://colab.research.google.com/github/Maxouuu4/test_git/blob/main/math0487_nb2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📓 Notebook 2 : **Modèle et vraisemblance**

## Objectifs 🎯

Les objectifs de ce notebook sont les suivants :
- *manipuler* les paramètres d'un modèle statistique ;
- *comprendre* le concept de vraisemblance et de log-vraisemblance d'un modèle, ainsi que les propriétés d'invariance associées.

## Contexte 🔍

Dans ce second notebook, nous allons aborder différents concepts liés aux modèles et à la vraisemblance à l'aide d'un ensemble de données disponible [ici](https://www.kaggle.com/datasets/mosapabdelghany/medical-insurance-cost-dataset). Les données concernent les maisons recensées dans un district donné de Californie et certaines statistiques sommaires les concernant, basées sur les données du recensement de 1990.

## Importation des librairies Python 🐍
Commençons par importer quelques librairies nécessaires pour la manipulation de modèles :
- [Scipy](https://docs.scipy.org/doc/scipy/index.html) étend NumPy et propose des algorithmes fondamentaux pour le calcul scientifique, ainsi qu'un riche module d'utilitaires pour la modélisation statistique.

In [ ]:
import pandas as pd
from scipy.stats import weibull_min, gamma, lognorm, norm, expon, multivariate_normal
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from mpl_toolkits.axes_grid1 import make_axes_locatable

## Chargement des données 🐼

Nous allons dans un premier temps charger les données depuis Kaggle et les stocker dans une `DataFrame` de la librairie pandas.

In [ ]:
df = pd.read_csv('/content/sample_data/california_housing_train.csv')
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


## Modélisation à partir d'un échantillon 🛠️
Dans le premier notebook, nous nous sommes attardés sur différentes manières de **décrire** les données dans un échantillon. Ces outils sont utiles, mais limités...

> En effet, ils ne permettent d'expliquer, prédire ou inférer que très peu de choses concernant le processus aléatoire qui a généré ces données, et nous aimerions aller un pas plus loin. En fonction du problème étudié, on voudrait par exemple pouvoir prédire quelles données ont le plus de chance d'être mesurées dans le futur, ou encore pouvoir tester des hypothèses sur le processus générateur (nous verrons ça plus tard, quand nous aborderons les tests d'hypothèses). 🤔

Afin de répondre à ces besoins, il nous faut **modéliser** le processus qui aurait généré nos données observées, et c'est donc tout naturellement que nous allons introduire un concept fondamental : le **modèle statistique**.

Dans ce notebook, nous allons nous concentrer sur les modèles paramétriques. Dans ce cadre, un modèle correspond à une famille de distributions de paramètre $\theta$ (potentiellement à plusieurs dimensions), où chaque valeur fixée de $\theta$ représente un membre de cette famille. Par exemple, le modèle normal pour la variable $Y$, $$Y \sim N(\mu, \sigma^2)$$ est de paramètre $\theta=(\mu, \sigma)$ non fixé, c'est la "famille". Si l'on fixe les paramètres en voulant coller aux données récoltées (nous verrons plus tard comment faire), on peut par exemple décider de modéliser le processur générateur par $$Y \sim N(\mu=2.4, \sigma^2=1.3^2),$$
c'est le "membre de la famille", à partir duquel on peut faire des prédictions par exemple.

> ⚠️ Nous avons ici parlé d'un modèle pour une variable $Y$, qui se cristallise par un point de donnée unique $y$. Évidemment, spécifier un modèle $f_Y(y)$ pour cette variable unique nous donne accès au modèle d'un échantillon *i.i.d.* $(Y_1, \ldots , Y_n)$, puisque dans ce cas $$f_{Y_1, \ldots , Y_n}(y_1,\ldots,y_n)=\prod_{j=1}^{n} f_{Y_j}(y_j).$$

Vous suivez toujours ? Parfait ! 💪

Voyons un peu comment nous pourrions modéliser la variable "median_house_value" de notre échantillon, et comment les paramètres influencent la forme de chaque modèle. 👀

In [ ]:
# Get the empirical data
median_house_value = df['median_house_value']
sorted_data = np.sort(median_house_value)
y_cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)


def plot_distribution_fit(distribution_name, **params):
    plt.figure(figsize=(16, 7)) # Increased figure size

    # Generate x-values for theoretical distribution plots
    # Use a wider range for x_values to cover potential tails of distributions
    x_values = np.linspace(median_house_value.min() - 50000, median_house_value.max() + 50000, 1000)
    x_values = x_values[x_values >= 0] # Ensure x_values are non-negative

    # Get the selected distribution
    dist = None
    if distribution_name == 'Normal':
        dist = norm(**params)
    elif distribution_name == 'Exponential':
        dist = expon(**params)
    elif distribution_name == 'Gamma':
        dist = gamma(**params)
    elif distribution_name == 'Log-Normal':
        dist = lognorm(**params)
    elif distribution_name == 'Weibull':
        dist = weibull_min(**params)
    else:
        print("Unsupported distribution")
        return

    # Plot Empirical PDF (Histogram) and Theoretical PDF
    plt.subplot(1, 2, 1)
    # Use 'hist' directly for more control over plotting, add edge color for clarity
    plt.hist(median_house_value, bins=70, density=True, alpha=0.7, color='#1f77b4', edgecolor='black', label='Histogram') # Refined color and bins
    plt.plot(x_values, dist.pdf(x_values), 'r-', lw=2.5, label=f'Theoretical PDF ({distribution_name})') # Thicker line
    plt.title('PDF: Histogram vs. Model', fontsize=14) # Added title and increased font size
    plt.xlabel('Median House Value', fontsize=12) # Increased font size
    plt.ylabel('Density', fontsize=12) # Increased font size
    plt.legend(fontsize=10) # Increased font size for legend
    plt.grid(True, linestyle='--', alpha=0.6) # Improved grid style
    plt.xlim([median_house_value.min() - 10000, median_house_value.max() + 10000]) # Adjust x-limits


    # Plot Empirical CDF and Theoretical CDF
    plt.subplot(1, 2, 2)
    plt.plot(sorted_data, y_cdf, marker='.', linestyle='none', markersize=3, color='#ff7f0e', label='Empirical CDF', alpha=0.7) # Refined color, marker size, alpha
    plt.plot(x_values, dist.cdf(x_values), 'g-', lw=2.5, label=f'Theoretical CDF ({distribution_name})') # Thicker line
    plt.title('CDF: Empirical vs. Theoretical Fit', fontsize=14) # Added title and increased font size
    plt.xlabel('Median House Value', fontsize=12) # Increased font size
    plt.ylabel('CDF', fontsize=12) # Increased font size
    plt.legend(fontsize=10) # Increased font size for legend
    plt.grid(True, linestyle='--', alpha=0.6) # Improved grid style
    plt.xlim([median_house_value.min() - 10000, median_house_value.max() + 10000]) # Adjust x-limits


    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjusted layout to make space for title if needed later
    plt.suptitle(f'Distribution Fit for Median House Value: {distribution_name}', fontsize=16, y=0.99) # Added a main title
    plt.show()

# Update dropdown options
distribution_selector = widgets.Dropdown(
    options=['Normal', 'Exponential', 'Gamma', 'Log-Normal', 'Weibull'],
    value='Normal',
    description='Distribution:'
)

# Function to update sliders based on selected distribution
def update_params_widgets(change):
    dist_name = change['new']
    if dist_name == 'Normal':
        param1_slider.description = 'Param 1 (loc):'
        param1_slider.min = 50000
        param1_slider.max = 500000
        param1_slider.step = 100
        param1_slider.value = median_house_value.mean()
        param2_slider.description = 'Param 2 (scale):'
        param2_slider.min = 500
        param2_slider.max = 200000
        param2_slider.step = 100
        param2_slider.value = median_house_value.std()
        param3_slider.description = 'Param 3 (shape):'
        param3_slider.disabled = True
        param3_slider.layout.visibility = "hidden"
    elif dist_name == 'Exponential':
        param1_slider.description = 'Param 1 (loc):'
        param1_slider.min = 0
        param1_slider.max = 500000
        param1_slider.step = 1000
        param1_slider.value = median_house_value.min() # Typically loc is min value
        param2_slider.description = 'Param 2 (scale):'
        param2_slider.min = 1
        param2_slider.max = 500000
        param2_slider.step = 1000
        param2_slider.value = median_house_value.mean() - median_house_value.min() # Typically scale is mean - loc
        param3_slider.description = 'Param 3 (shape):'
        param3_slider.disabled = True
        param3_slider.layout.visibility = "hidden"
    elif dist_name == 'Gamma':
        param1_slider.description = 'Param 1 (a):'
        param1_slider.min = 0.1
        param1_slider.max = 10
        param1_slider.step = 0.1
        param1_slider.value = 1 # Example value
        param2_slider.description = 'Param 2 (loc):'
        param2_slider.min = 0
        param2_slider.max = 500000
        param2_slider.step = 1000
        param2_slider.value = 0 # Example value
        param3_slider.description = 'Param 3 (scale):'
        param3_slider.min = 1
        param3_slider.max = 200000
        param3_slider.step = 1000
        param3_slider.value = 100000 # Example value
        param3_slider.disabled = False
        param3_slider.layout.visibility = "visible"
    elif dist_name == 'Log-Normal':
        param1_slider.description = 'Param 1 (s):'
        param1_slider.min = 0.1
        param1_slider.max = 5
        param1_slider.step = 0.1
        param1_slider.value = 1 # Example value
        param2_slider.description = 'Param 2 (loc):'
        param2_slider.min = 0
        param2_slider.max = 500000
        param2_slider.step = 1000
        param2_slider.value = 0 # Example value
        param3_slider.description = 'Param 3 (scale):'
        param3_slider.min = 1
        param3_slider.max = 500000
        param3_slider.step = 1000
        param3_slider.value = np.exp(np.mean(np.log(median_house_value[median_house_value > 0]))) # Example value (geometric mean)
        param3_slider.disabled = False
        param3_slider.layout.visibility = "visible"
    elif dist_name == 'Weibull':
        param1_slider.description = 'Param 1 (c):' # Shape parameter
        param1_slider.min = 0.1
        param1_slider.max = 5
        param1_slider.step = 0.1
        param1_slider.value = 1 # Example value
        param2_slider.description = 'Param 2 (loc):'
        param2_slider.min = 0
        param2_slider.max = 500000
        param2_slider.step = 1000
        param2_slider.value = median_house_value.min() # Typically loc is min value
        param3_slider.description = 'Param 3 (scale):'
        param3_slider.min = 1
        param3_slider.max = 500000
        param3_slider.step = 1000
        param3_slider.value = median_house_value.std() # Example value
        param3_slider.disabled = False
        param3_slider.layout.visibility = "visible"

distribution_selector.observe(update_params_widgets, names='value')

# Link widgets to the plotting function
def interactive_plot(distribution_name, param1, param2, param3):
    params = {}
    if distribution_name == 'Normal':
        params = {'loc': param1, 'scale': param2}
    elif distribution_name == 'Exponential':
         params = {'loc': param1, 'scale': param2}
    elif distribution_name == 'Gamma':
        params = {'a': param1, 'loc': param2, 'scale': param3}
    elif distribution_name == 'Log-Normal':
        params = {'s': param1, 'loc': param2, 'scale': param3}
    elif distribution_name == 'Weibull':
        params = {'c': param1, 'loc': param2, 'scale': param3}
    plot_distribution_fit(distribution_name, **params)


param1_slider = widgets.FloatSlider(min=50000, max=500000, step=100, value=median_house_value.mean(), description='Param 1 (loc):')
param2_slider = widgets.FloatSlider(min=500, max=200000, step=100, value=median_house_value.std(), description='Param 2 (scale):')
param3_slider = widgets.FloatSlider(description = 'Param 3 (shape):')
param3_slider.disabled = True
param3_slider.layout.visibility = "hidden"

interactive_widget = widgets.interactive(
    interactive_plot,
    distribution_name=distribution_selector,
    param1=param1_slider,
    param2=param2_slider,
    param3=param3_slider
)

# Display the widgets and the plot
display(interactive_widget)

interactive(children=(Dropdown(description='Distribution:', options=('Normal', 'Exponential', 'Gamma', 'Log-No…

Vous avez pu sélectionner divers modèles et, pour chacun de ceux-ci, jouer avec ses paramètres.

❓Que représente le graphe de gauche ? Cela a-t-il du sens de superposer la courbe rouge à l'histogramme bleu ?

❓Que manipulez-vous comme paramètres ? Avez-vous vérifié que les paramètres des différentes distributions, définis par Scipy, correspondent à ceux définis dans le cours de MATH0487 ? Quelle influence des définitions différentes peuvent-elles avoir ?

❓Quelle distribution et pour quels paramètres trouvez-vous que le modèle correspond le mieux aux données récoltées ? Trouvez-vous que ce modèle est satisfaisant ? Quid du "pic" à droite de l'histogramme ? À quoi peut-il être dû ? Doit-on chercher à le prendre en compte dans notre modélisation ?

❓Que représente le graphe de droite ? Cela a-t-il du sens de superposer la courbe verte avec l'ECDF orange ? Un bon "fit" sur la figure de gauche correspond-il à un bon "fit" sur la figure de droite ?

> 🧠 Écrivez ici ce que vous en pensez !

Nous n'avons tant que maintenant exploré que des modèles unimodaux, c'est-à-dire ne contenant qu'un seul "pic". Évidemment, il est possible de combiner de tels modèles pour en obtenir de nouveaux, et ainsi pouvoir modéliser des données qui seraient distribuées autour de plusieurs pics.

Voyons à quoi cela ressemble pour la variable "longitude" de notre échantillon, que nous allons modéliser avec une mixture de Normales. 👀

In [ ]:
# Load the dataframe
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

# Get the longitude data
longitude_data = df['longitude']
sorted_longitude_data = np.sort(longitude_data)
y_cdf_longitude = np.arange(1, len(sorted_longitude_data) + 1) / len(sorted_longitude_data)


# Define the PDF and CDF of a mixture of two Gaussian distributions
def gaussian_mixture_pdf(x, mu1, std1, mu2, std2, mix_coeff):
    """
    Calculates the PDF of a mixture of two Gaussian distributions.

    Args:
        x (numpy.ndarray): The input values.
        mu1 (float): Mean of the first Gaussian.
        std1 (float): Standard deviation of the first Gaussian.
        mu2 (float): Mean of the second Gaussian.
        std2 (float): Standard deviation of the second Gaussian.
        mix_coeff (float): Mixing coefficient for the first Gaussian (between 0 and 1).

    Returns:
        numpy.ndarray: The PDF values at x.
    """
    # Ensure standard deviations are positive to avoid errors
    std1 = max(std1, 1e-6)
    std2 = max(std2, 1e-6)
    return mix_coeff * norm.pdf(x, loc=mu1, scale=std1) + (1 - mix_coeff) * norm.pdf(x, loc=mu2, scale=std2)

def gaussian_mixture_cdf(x, mu1, std1, mu2, std2, mix_coeff):
    """
    Calculates the CDF of a mixture of two Gaussian distributions.

    Args:
        x (numpy.ndarray): The input values.
        mu1 (float): Mean of the first Gaussian.
        std1 (float): Standard deviation of the first Gaussian.
        mu2 (float): Mean of the second Gaussian.
        std2 (float): Standard deviation of the second Gaussian.
        mix_coeff (float): Mixing coefficient for the first Gaussian (between 0 and 1).

    Returns:
        numpy.ndarray: The CDF values at x.
    """
    # Ensure standard deviations are positive to avoid errors
    std1 = max(std1, 1e-6)
    std2 = max(std2, 1e-6)
    return mix_coeff * norm.cdf(x, loc=mu1, scale=std1) + (1 - mix_coeff) * norm.cdf(x, loc=mu2, scale=std2)


# Create interactive sliders for the parameters
mu1_slider = widgets.FloatSlider(min=-125, max=-114, step=0.1, value=-122, description='Mean 1:')
std1_slider = widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1, description='Std Dev 1:')
mu2_slider = widgets.FloatSlider(min=-125, max=-114, step=0.1, value=-117, description='Mean 2:')
std2_slider = widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1, description='Std Dev 2:')
mix_coeff_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.5, description='Mix Coeff:')


# Function to plot the histogram/PDF and the CDF
def plot_mixture_fit(mu1, std1, mu2, std2, mix_coeff):
    plt.figure(figsize=(16, 6)) # Increased figure size for two subplots

    # Plot the empirical histogram and theoretical PDF
    plt.subplot(1, 2, 1) # 1 row, 2 columns, 1st plot
    plt.hist(longitude_data, bins=50, density=True, alpha=0.6, color='g', edgecolor='black', label='Empirical Histogram')

    # Generate x-values for the theoretical PDF and CDF
    x_values = np.linspace(longitude_data.min(), longitude_data.max(), 500)

    # Plot the theoretical PDF of the mixture model
    pdf_values = gaussian_mixture_pdf(x_values, mu1, std1, mu2, std2, mix_coeff)
    plt.plot(x_values, pdf_values, 'r-', lw=2, label='Gaussian Mixture Model PDF')

    plt.title('PDF: Histogram vs. Gaussian Mixture Fit')
    plt.xlabel('Longitude')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)


    # Plot Empirical and Theoretical CDF
    plt.subplot(1, 2, 2) # 1 row, 2 columns, 2nd plot
    plt.plot(sorted_longitude_data, y_cdf_longitude, marker='.', linestyle='none', markersize=3, color='#ff7f0e', label='Empirical CDF', alpha=0.7)

    # Plot the theoretical CDF of the mixture model
    cdf_values = gaussian_mixture_cdf(x_values, mu1, std1, mu2, std2, mix_coeff)
    plt.plot(x_values, cdf_values, 'g-', lw=2.5, label='Theoretical CDF (Gaussian Mixture)')

    plt.title('CDF: Empirical vs. Gaussian Mixture Fit')
    plt.xlabel('Longitude')
    plt.ylabel('CDF')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)


    plt.tight_layout() # Adjust layout to prevent overlapping titles/labels
    plt.show()

# Link widgets to the plotting function
interactive_plot_mixture = widgets.interactive(
    plot_mixture_fit,
    mu1=mu1_slider,
    std1=std1_slider,
    mu2=mu2_slider,
    std2=std2_slider,
    mix_coeff=mix_coeff_slider
)

# Display the widgets and the plot
display(interactive_plot_mixture)

interactive(children=(FloatSlider(value=-122.0, description='Mean 1:', max=-114.0, min=-125.0), FloatSlider(va…

Plus pratique qu'une distribution normale à un seul lobe, n'est-ce pas ? 😉

❓Comment est définie la mixture de Normales ? Que représentent les différents paramètres que vous pouvez contrôler ? Pourrait-on n'avoir qu'un seul curseur pour le paramètre "Mix Coeff" si nous mixions trois Normales ?

> 🧠 Écrivez ici ce que vous en pensez !

Bon, nous pouvons désormais modéliser une variable unimodale, une variable bimodale... mais quid de deux variables ? Comment pouvons-nous modéliser la distribution jointe de deux variables de notre échantillon ? 🤔

En utilisant une distribution multivariée, pardi !

Voyons voir à quoi cela ressemble pour une distribution normale à deux variables. 👀

In [ ]:
# Load the dataframe
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

# Extract households and median_income data
households_data = df['households']
median_income_data = df['median_income']

# Calculate empirical CDF for marginals
sorted_households = np.sort(households_data)
y_cdf_households = np.arange(1, len(sorted_households) + 1) / len(sorted_households)

sorted_median_income = np.sort(median_income_data)
y_cdf_median_income = np.arange(1, len(sorted_median_income) + 1) / len(sorted_median_income)


# Define a 2D Gaussian PDF and its marginals
def bivariate_normal_pdf(x, y, mean, cov):
    """Calculates the PDF of a bivariate normal distribution."""
    pos = np.dstack((x, y))
    return multivariate_normal.pdf(pos, mean=mean, cov=cov)

def bivariate_normal_marginal_pdf_x(x, mean, cov):
    """Calculates the marginal PDF for the first variable (households)."""
    return norm.pdf(x, loc=mean[0], scale=np.sqrt(cov[0][0]))

def bivariate_normal_marginal_pdf_y(y, mean, cov):
    """Calculates the marginal PDF for the second variable (median_income)."""
    return norm.pdf(y, loc=mean[1], scale=np.sqrt(cov[1][1]))

def bivariate_normal_marginal_cdf_x(x, mean, cov):
    """Calculates the marginal CDF for the first variable (households)."""
    return norm.cdf(x, loc=mean[0], scale=np.sqrt(cov[0][0]))

def bivariate_normal_marginal_cdf_y(y, mean, cov):
    """Calculates the marginal CDF for the second variable (median_income)."""
    return norm.cdf(y, loc=mean[1], scale=np.sqrt(cov[1][1]))


# Create widgets
visualization_selector = widgets.Dropdown(
    options=['Scatter Plot', 'Heatmap'],
    value='Scatter Plot',
    description='2D Data Viz:'
)

# Initial parameters for the 2D Gaussian (can be expanded for more complex models)
initial_mean = [households_data.mean(), median_income_data.mean()]
initial_cov = np.cov(households_data, median_income_data)

mean_x_slider = widgets.FloatSlider(min=0, max=households_data.max() * 1.2, step=10, value=initial_mean[0], description='Mean (Households):')
mean_y_slider = widgets.FloatSlider(min=0, max=median_income_data.max() * 1.2, step=0.1, value=initial_mean[1], description='Mean (Median Income):')

# For simplicity, let's initially use diagonal covariance for interactive sliders
# A full covariance matrix is harder to control with simple sliders
std_dev_x_slider = widgets.FloatSlider(min=1, max=households_data.std() * 2, step=10, value=np.sqrt(initial_cov[0,0]), description='Std Dev (Households):')
std_dev_y_slider = widgets.FloatSlider(min=0.01, max=median_income_data.std() * 2, step=0.01, value=np.sqrt(initial_cov[1,1]), description='Std Dev (Median Income):')
# You could add a correlation/covariance term slider if needed for full covariance

# Function to create and update the plot
def update_2d_plot(viz_type, mean_x, mean_y, std_dev_x, std_dev_y):
    plt.figure(figsize=(12, 10)) # Adjusted figure size for marginals
    gs = plt.GridSpec(2, 2, width_ratios=[4, 1], height_ratios=[1, 4])
    ax_scatter = plt.subplot(gs[1, 0])
    ax_hist_x = plt.subplot(gs[0, 0], sharex=ax_scatter)
    ax_hist_y = plt.subplot(gs[1, 1], sharey=ax_scatter)
    ax_cdf_x = ax_hist_x.twinx() # Twin axes for CDF on histogram plot
    ax_cdf_y = ax_hist_y.twiny() # Twin axes for CDF on histogram plot

    # Disable axis labels and ticks for marginal histograms/CDFs initially
    ax_hist_x.tick_params(axis='x', labelbottom=False)
    ax_hist_y.tick_params(axis='y', labelleft=False)


    # --- 2D Data Visualization (Scatter or Heatmap) ---
    if viz_type == 'Scatter Plot':
        ax_scatter.scatter(households_data, median_income_data, s=1, alpha=0.5, label='Data Points')
        ax_scatter.set_title('2D Data and Theoretical Model')
    elif viz_type == 'Heatmap':
        # Use histogram2d for heatmap
        hist, xedges, yedges = np.histogram2d(households_data, median_income_data, bins=50)
        im = ax_scatter.imshow(hist.T, extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], origin='lower', cmap='viridis', aspect='auto')
        ax_scatter.set_title('2D Heatmap and Theoretical Model')
        # Add colorbar
        divider = make_axes_locatable(ax_scatter)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im, cax=cax, label='Count')


    # --- Theoretical 2D Model Contours (Bivariate Normal) ---
    # Create a grid for the theoretical PDF
    households_min, households_max = households_data.min(), households_data.max()
    median_income_min, median_income_max = median_income_data.min(), median_income_data.max()
    households_grid = np.linspace(households_min, households_max, 100)
    median_income_grid = np.linspace(median_income_min, median_income_max, 100)
    X, Y = np.meshgrid(households_grid, median_income_grid)

    # Use diagonal covariance matrix from sliders
    current_cov = [[std_dev_x**2, 0], [0, std_dev_y**2]]
    current_mean = [mean_x, mean_y]

    Z = bivariate_normal_pdf(X, Y, mean=current_mean, cov=current_cov)
    ax_scatter.contour(X, Y, Z, levels=10, colors='r', alpha=0.7, linewidths=1.5) # Equi-probability contours


    ax_scatter.set_xlabel('Households')
    ax_scatter.set_ylabel('Median Income')
    ax_scatter.grid(True, linestyle='--', alpha=0.6)
    ax_scatter.legend(loc='upper left')


    # --- 1D Marginal Histograms and Empirical CDFs ---
    # Households marginal
    ax_hist_x.hist(households_data, bins=50, density=True, alpha=0.6, color='g', edgecolor='black', label='Empirical Hist (Households)')
    ax_cdf_x.plot(sorted_households, y_cdf_households, marker='.', linestyle='none', markersize=3, color='#ff7f0e', label='Empirical CDF (Households)', alpha=0.7)
    ax_hist_x.set_ylabel('Density (Households)')
    ax_cdf_x.set_ylabel('CDF (Households)')

    # Median Income marginal
    ax_hist_y.hist(median_income_data, bins=50, density=True, alpha=0.6, color='g', edgecolor='black', orientation='horizontal', label='Empirical Hist (Median Income)')
    ax_cdf_y.plot(y_cdf_median_income, sorted_median_income, marker='.', linestyle='none', markersize=3, color='#ff7f0e', label='Empirical CDF (Median Income)', alpha=0.7)
    ax_hist_y.set_xlabel('Density (Median Income)')
    ax_cdf_y.set_xlabel('CDF (Median Income)')
    ax_hist_y.legend(loc='upper right')

    # --- Theoretical 1D Marginal PDFs and CDFs ---
    # Households marginal theoretical
    ax_hist_x.plot(households_grid, bivariate_normal_marginal_pdf_x(households_grid, mean=current_mean, cov=current_cov), 'r-', lw=2, label='Theoretical PDF (Households)')
    ax_cdf_x.plot(households_grid, bivariate_normal_marginal_cdf_x(households_grid, mean=current_mean, cov=current_cov), 'b-', lw=2, label='Theoretical CDF (Households)')
    ax_cdf_x.legend(loc='lower right')
    ax_hist_x.legend(loc='upper right')

    # Median Income marginal theoretical
    ax_hist_y.plot(bivariate_normal_marginal_pdf_y(median_income_grid, mean=current_mean, cov=current_cov), median_income_grid, 'r-', lw=2, label='Theoretical PDF (Median Income)')
    ax_cdf_y.plot(bivariate_normal_marginal_cdf_y(median_income_grid, mean=current_mean, cov=current_cov), median_income_grid, 'b-', lw=2, label='Theoretical CDF (Median Income)')
    ax_cdf_y.legend(loc='lower right')
    ax_hist_y.legend(loc='upper right')

    # Adjust layout and display
    plt.tight_layout()
    plt.show()

# Link widgets to the plotting function
interactive_plot_2d = widgets.interactive(
    update_2d_plot,
    viz_type=visualization_selector,
    mean_x=mean_x_slider,
    mean_y=mean_y_slider,
    std_dev_x=std_dev_x_slider,
    std_dev_y=std_dev_y_slider
)

# Display the widgets and the plot
display(interactive_plot_2d)

interactive(children=(Dropdown(description='2D Data Viz:', options=('Scatter Plot', 'Heatmap'), value='Scatter…

## Vraisemblance d'un modèle 📈

Maintenant que vous avez compris la notion de modèle, et que vous avez pu être témoins de l'influence des paramètres sur la forme des distributions, une question légitime se pose : comment, pour un modèle donné, juger de sa pertinence en fonction de ses paramètres ? Autrement dit, comment mesurer la plausibilité des paramètres d'un modèle au vu des données récoltées ? À quel point est-ce "probable" que mon modèle muni de ses paramètres soit à l'origine de la génération des données que j'ai recueillies ? 🤔

Pour répondre à cette question, introduisons un second concept fondamental en statistiques : la **fonction de vraisemblance**.

> Pour un modèle donné, la vraisemblance est une fonction des paramètres du modèle, qui mesure la plausibilité d'observer les données recueillies $(y_1, \ldots, y_n)$, et qu'on note
$$L(\theta; y_1, \ldots, y_n) = f_{Y_1, \ldots, Y_n}(y_1, \ldots, y_n; \theta)$$
où $f_Y( \cdot ; \theta)$ est le modèle choisi pour la variable $Y$. Ainsi, pour un échantillon *i.i.d.*, la vraisemblance s'écrit
$$L(\theta; y_1, \ldots, y_n) = \prod_{j=1}^{n} f_{Y_j}(y_j; \theta)$$
où chaque $f_{Y_j}(y_j; \theta)$ représente la contribution de la donnée $y_j$.

Voyons à quoi ressemble cette fonction, pour un sous-échantillon dont vous pouvez contrôler la taille, et pour un modèle normal (avec $\sigma$ fixé). 👀


In [ ]:

# Load the dataframe
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

# Get the empirical data
median_house_value = df['median_house_value']

# Create a button to generate a subsample
sample_size_slider = widgets.IntSlider(min=100, max=len(median_house_value), step=100, description='Sample Size:')
generate_sample_button = widgets.Button(description="Generate Subsample")
output_area = widgets.Output()

def generate_and_plot_sample(b):
    with output_area:
        output_area.clear_output()
        sample_data = median_house_value.sample(n=sample_size_slider.value).values
        sorted_sample_data = np.sort(sample_data)

        # Example: Fit a Normal distribution (single parameter: loc, assuming scale is fixed for simplicity)
        # In a real scenario, you might fit the parameter using MLE or other methods.
        # For this example, we'll use the sample mean as the 'loc' parameter.
        sample_mean = np.mean(sample_data)
        scale_param = np.std(median_house_value) # Use overall standard deviation for simplicity

        # Define the log-likelihood function for the Normal distribution (simplified for 'loc')
        def normal_likelihood(loc, data, scale):
            # The log-likelihood is the sum of the log of the PDFs
            return np.prod(norm.pdf(data, loc=loc, scale=scale))

        # Generate parameter values for likelihood plot
        loc_values = np.linspace(sample_mean - 50000, sample_mean + 50000, 200)
        log_likelihood_values = [normal_likelihood(loc, sample_data, scale_param) for loc in loc_values]

        # Create parameter slider for the plot
        param_slider = widgets.FloatSlider(
            min=loc_values.min(),
            max=loc_values.max(),
            step=(loc_values.max() - loc_values.min()) / 100,
            value=sample_mean,
            description='Mean (loc):'
        )

        def plot_sample_fit_and_likelihood(loc_param):
            plt.figure(figsize=(16, 6))

            # Plot Histogram and Theoretical PDF
            plt.subplot(1, 2, 1)
            plt.hist(sample_data, bins=30, density=True, alpha=0.7, color='#1f77b4', edgecolor='black', label='Sample Histogram')
            x_values = np.linspace(sample_data.min() - 10000, sample_data.max() + 10000, 200)
            x_values = x_values[x_values >= 0]
            plt.plot(x_values, norm.pdf(x_values, loc=loc_param, scale=scale_param), 'r-', lw=2.5, label=f'Theoretical PDF (Normal, loc={loc_param:.2f})')
            plt.title('Sample Histogram vs. Theoretical PDF Fit')
            plt.xlabel('Median House Value')
            plt.ylabel('Density')
            plt.legend()
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.xlim([sample_data.min() - 5000, sample_data.max() + 5000])


            # Plot Log-Likelihood Function
            plt.subplot(1, 2, 2)
            plt.plot(loc_values, log_likelihood_values, 'b-', lw=2, label='Likelihood Function')

            # Find the log-likelihood value for the selected loc_param
            # Find the index in loc_values closest to loc_param
            closest_index = np.abs(loc_values - loc_param).argmin()
            selected_log_likelihood_value = log_likelihood_values[closest_index]


            plt.axvline(x=loc_param, color='red', linestyle='--', lw=2, label=f'Selected loc: {loc_param:.2f}, Likelihood: {selected_log_likelihood_value:.2f}')
            plt.plot(loc_param, selected_log_likelihood_value, 'ro', markersize=8) # Add a red dot at the selected point


            plt.title('Likelihood Function for Normal Distribution (fixed scale)')
            plt.xlabel('Mean (loc)')
            plt.ylabel('Log-Likelihood')
            plt.legend()
            plt.grid(True, linestyle='--', alpha=0.6)

            plt.tight_layout()
            plt.show()

        interactive_plot_sample = widgets.interactive(
            plot_sample_fit_and_likelihood,
            loc_param=param_slider
        )
        display(interactive_plot_sample)

generate_sample_button.on_click(generate_and_plot_sample)

display(sample_size_slider, generate_sample_button, output_area)

IntSlider(value=100, description='Sample Size:', max=17000, min=100, step=100)

Button(description='Generate Subsample', style=ButtonStyle())

Output()

🤔 Mmh le graphe de droite n'est pas fort lisible... il semblerait que la vraisemblance soit nulle pour toutes les valeurs du paramètre $\mu$ ...

> Rappelons-nous comment la vraisemblance est calculée : elle implique le produit des densités marginales $f_{Y_j}(y_j; \theta)$. Ces valeurs pouvant être très petites, en multiplier une centaine donne un résultat si petit qu'il est considéré comme nul par l'ordinateur ! N'hésitez pas à modifier la taille d'échantillon minimale du curseur, pour des tailles faibles, vous verrez que la courbe de droite est plus indicative ! 🔍

Ceci étant dit, que faire si nous avons un échantillon de grande taille ? Une manière très simple de résoudre ce genre de souci, et qui en plus facilite les développements, est de considérer le logarithme de la vraisemblance ! Cette fonction est alors appelée log-vraisemblance, tout simplement, et se note $$l(\theta; \mathbf{y}) = \log L(\theta; \mathbf{y}),$$ où $\mathbf{y}=y_1, \ldots, y_n$.

Elle a l'avantage de transformer le produit des marginales en une somme de logarithmes.

Reproduisons ainsi les mêmes visualisations que précédemment, mais cette-fois ci en considérant la log-vraisemblance. 👀

In [ ]:
# Load the dataframe
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

# Get the empirical data
median_house_value = df['median_house_value']

# Create a button to generate a subsample
sample_size_slider = widgets.IntSlider(min=100, max=len(median_house_value), step=100, description='Sample Size:')
generate_sample_button = widgets.Button(description="Generate Subsample")
output_area = widgets.Output()

def generate_and_plot_sample(b):
    with output_area:
        output_area.clear_output()
        sample_data = median_house_value.sample(n=sample_size_slider.value).values
        sorted_sample_data = np.sort(sample_data)

        # Example: Fit a Normal distribution (single parameter: loc, assuming scale is fixed for simplicity)
        # In a real scenario, you might fit the parameter using MLE or other methods.
        # For this example, we'll use the sample mean as the 'loc' parameter.
        sample_mean = np.mean(sample_data)
        scale_param = np.std(median_house_value) # Use overall standard deviation for simplicity

        # Define the log-likelihood function for the Normal distribution (simplified for 'loc')
        def normal_log_likelihood(loc, data, scale):
            # The log-likelihood is the sum of the log of the PDFs
            return np.sum(norm.logpdf(data, loc=loc, scale=scale))

        # Generate parameter values for likelihood plot
        loc_values = np.linspace(sample_mean - 50000, sample_mean + 50000, 200)
        log_likelihood_values = [normal_log_likelihood(loc, sample_data, scale_param) for loc in loc_values]

        # Create parameter slider for the plot
        param_slider = widgets.FloatSlider(
            min=loc_values.min(),
            max=loc_values.max(),
            step=(loc_values.max() - loc_values.min()) / 100,
            value=sample_mean,
            description='Mean (loc):'
        )

        def plot_sample_fit_and_likelihood(loc_param):
            plt.figure(figsize=(16, 6))

            # Plot Histogram and Theoretical PDF
            plt.subplot(1, 2, 1)
            plt.hist(sample_data, bins=30, density=True, alpha=0.7, color='#1f77b4', edgecolor='black', label='Sample Histogram')
            x_values = np.linspace(sample_data.min() - 10000, sample_data.max() + 10000, 200)
            x_values = x_values[x_values >= 0]
            plt.plot(x_values, norm.pdf(x_values, loc=loc_param, scale=scale_param), 'r-', lw=2.5, label=f'Theoretical PDF (Normal, loc={loc_param:.2f})')
            plt.title('Sample Histogram vs. Theoretical PDF Fit')
            plt.xlabel('Median House Value')
            plt.ylabel('Density')
            plt.legend()
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.xlim([sample_data.min() - 5000, sample_data.max() + 5000])


            # Plot Log-Likelihood Function
            plt.subplot(1, 2, 2)
            plt.plot(loc_values, log_likelihood_values, 'b-', lw=2, label='Log-Likelihood Function')

            # Find the log-likelihood value for the selected loc_param
            # Find the index in loc_values closest to loc_param
            closest_index = np.abs(loc_values - loc_param).argmin()
            selected_log_likelihood_value = log_likelihood_values[closest_index]


            plt.axvline(x=loc_param, color='red', linestyle='--', lw=2, label=f'Selected loc: {loc_param:.2f}, Log-Likelihood: {selected_log_likelihood_value:.2f}')
            plt.plot(loc_param, selected_log_likelihood_value, 'ro', markersize=8) # Add a red dot at the selected point


            plt.title('Log-Likelihood Function for Normal Distribution (fixed scale)')
            plt.xlabel('Mean (loc)')
            plt.ylabel('Log-Likelihood')
            plt.legend()
            plt.grid(True, linestyle='--', alpha=0.6)

            plt.tight_layout()
            plt.show()

        interactive_plot_sample = widgets.interactive(
            plot_sample_fit_and_likelihood,
            loc_param=param_slider
        )
        display(interactive_plot_sample)

generate_sample_button.on_click(generate_and_plot_sample)

display(sample_size_slider, generate_sample_button, output_area)

IntSlider(value=100, description='Sample Size:', max=17000, min=100, step=100)

Button(description='Generate Subsample', style=ButtonStyle())

Output()

😀 C'est mieux, non ?

On remarque donc que la log-vraisemblance, pour un modèle normal avec $\sigma$ fixé, décrit une sorte de cloche concave en fonction du paramètre $\mu$. Ainsi, à chaque valeur de $\mu$ correspond une valeur de log-vraisemblance (mise en évidence avec un cercle rouge), qui mesure (de manière logarithmique) à quel point le modèle paramétrisé avec cette valeur de $\mu$ explique les données observées, c'est-à-dire à quel point il est vraisemblabe que ce modèle paramétrisé ait généré ces données.

❓Cela a-t-il du sens de superposer l'histogramme et la courbe rouge ? Pourquoi ?

❓Que remarquez-vous lorsque vous placez le paramètre au maximum de la courbe de log-vraisemblance ?

❓Le paramètre donnant le maximum de cette courbe est-il toujours celui donnant le maximum de la courbe de vraisemblance ? À quoi cela sert-il de déterminer le paramètre donnant le maximum de cette mesure ?

❓Est-ce toujours une opération valable de calculer le logarithme de la fonction de vraisemblance ?

❓À quoi peut servir le *likelihood-ratio test*, qui repose sur l'évaluation du ratio des vraisemblances de deux modèles paramétrés ?

❓La vraisemblance en une valeur de paramètre donnée correspond-elle à une valeur de probabilité ? Cela dépend-il de la nature des données ?

> 🧠 Écrivez ici ce que vous en pensez !

Afin de bien saisir la contribution de chaque élément d'un échantillon dans la vraisemblance d'un modèle pour cet échantillon, nous vous proposons de visualiser l'influence de ces éléments de la distribution marginale associée ci-dessous. 👀


In [ ]:
# Load the dataframe
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

# Get the empirical data (using median_house_value as in the previous log-likelihood example)
empirical_data = df['median_house_value']

# Create a sample size slider
sample_size_slider = widgets.IntSlider(min=1, max=10, step=1, description='Sample Size:')

# Define a theoretical distribution (using Normal for simplicity)
def theoretical_pdf(x, loc, scale):
    return norm.pdf(x, loc=loc, scale=scale)

def theoretical_logpdf(x, loc, scale):
    # Ensure scale is positive to avoid errors
    scale = max(scale, 1e-6)
    return norm.logpdf(x, loc=loc, scale=scale)

# Initial parameters for the theoretical distribution (using the full dataset's std for simplicity)
initial_loc = empirical_data.mean()
initial_scale = empirical_data.std()

# Generate parameter values for likelihood plot (same range as the slider)
# Extend the range slightly to show the curves beyond the slider's limits if needed
loc_values_for_plot = np.linspace(empirical_data.min() - 100000, empirical_data.max() + 100000, 500)
loc_values_for_plot = loc_values_for_plot[loc_values_for_plot >= 0] # Ensure positive values if applicable


# Create parameter slider for the theoretical distribution (e.g., mean for Normal)
loc_slider = widgets.FloatSlider(
    min=loc_values_for_plot.min(), # Use the wider range for the slider as well
    max=loc_values_for_plot.max(),
    step=(loc_values_for_plot.max() - loc_values_for_plot.min()) / 200,
    value=initial_loc,
    description='Model Mean (loc):'
)


# Function to generate a sample and update the plot
def update_sample_and_plot(sample_size, loc):
    plt.figure(figsize=(16, 6))

    # Generate a random sample
    sample_data = empirical_data.sample(n=sample_size, replace=False).values # Use replace=False for sampling without replacement

    # --- Plot Histogram and Theoretical PDF ---
    plt.subplot(1, 2, 1)
    # Adjust bins based on sample size, but with a reasonable maximum
    bins = min(sample_size, 50)
    plt.hist(sample_data, bins=bins, density=True, alpha=0.7, color='#1f77b4', edgecolor='black', label='Sample Histogram')
    x_values_pdf = np.linspace(sample_data.min() - 10000, sample_data.max() + 10000, 200)
    x_values_pdf = x_values_pdf[x_values_pdf >= 0]
    plt.plot(x_values_pdf, theoretical_pdf(x_values_pdf, loc=loc, scale=initial_scale), 'r-', lw=2.5, label=f'Theoretical PDF (Normal, loc={loc:.2f})')
    plt.title('Sample Histogram vs. Theoretical PDF Fit')
    plt.xlabel('Median House Value')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    # Adjust x-limits based on sample data range
    plt.xlim([sample_data.min() - 5000, sample_data.max() + 5000])


    # --- Plot Log-Likelihoods ---
    plt.subplot(1, 2, 2)

    # Calculate and plot the log-likelihood function for each sample point
    individual_log_likelihood_values = np.array([theoretical_logpdf(sample_point, loc_values_for_plot, initial_scale) for sample_point in sample_data])

    for i, log_likelihood_curve in enumerate(individual_log_likelihood_values):
        plt.plot(loc_values_for_plot, log_likelihood_curve, label=f'Sample {i+1} Log-Likelihood', alpha=0.7)

    # Calculate and plot the total log-likelihood function
    total_log_likelihood_curve = np.sum(individual_log_likelihood_values, axis=0)
    plt.plot(loc_values_for_plot, total_log_likelihood_curve, 'k-', lw=3, label='Total Log-Likelihood')

    # Mark the currently selected parameter value on the plot
    # Find the index in loc_values_for_plot closest to the current loc
    closest_index = np.abs(loc_values_for_plot - loc).argmin()
    selected_total_log_likelihood = total_log_likelihood_curve[closest_index]

    plt.axvline(x=loc, color='red', linestyle='--', lw=2, label=f'Selected loc: {loc:.2f}, Total Log-Likelihood: {selected_total_log_likelihood:.2f}')
    plt.plot(loc, selected_total_log_likelihood, 'ro', markersize=8) # Add a red dot at the selected point


    plt.title('Log-Likelihood Functions (Individual and Total)')
    plt.xlabel('Model Mean (loc)')
    plt.ylabel('Log-Likelihood')
    # Adjust legend to handle potentially many individual curves
    plt.legend(fontsize=8, loc='best')
    plt.grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.show()

# Link widgets to the plotting function
interactive_plot_likelihood = widgets.interactive(
    update_sample_and_plot,
    sample_size=sample_size_slider,
    loc=loc_slider
)

# Display the widgets and the plot
display(interactive_plot_likelihood)

interactive(children=(IntSlider(value=1, description='Sample Size:', max=10, min=1), FloatSlider(value=207300.…

❓Dans le cas d'un échantillon qui ne serait pas *i.i.d.*, ces développements et visualisations seraient-ils toujours pertinents ? Que signifie exactement cette hypothèse ? Est-elle toujours/parfois vérifiée ?

> 🧠 Écrivez ici ce que vous en pensez !

## Propriétés d'invariance de la vraisemblance ✅

Bien, maintenant que vous maîtrisez les concepts de modèle et de (log-)vraisemblance, illustrons deux propriétés plutôt utiles de cette dernière ! 👇

La première est l'invariance de la vraisemblance par rapport à la **reparamétrisation**. Autrement dit, si l'on souhaite utiliser une convention différente pour le paramètre $\theta$ de ce modèle en définissant $$\psi = g(\theta),$$ où $g(\cdot)$ est une fonction injective, la fonction de vraisemblance en fonction de $\psi$, $L_{\psi}(\psi; \mathbf{y})$, aura certes une forme différente de la fonction de vraisemblance en fonction de $\theta$, $L_{\theta}(\theta; \mathbf{y})$, mais à toute valeur $\theta_0$ correspond une valeur $\psi_0=g(\theta_0)$ telle que $$L_{\theta}(\theta_0; \mathbf{y})=L_{\psi}(\psi_0; \mathbf{y}).$$

Illustrons cela en reparamétrisant un modèle exponentiel pour la variable "median_income" de notre échantillon. 👀

In [ ]:
# Load the dataframe
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

# Get the median_income data
data = df['median_income']

# Define the log-likelihood function for the Exponential distribution
# Using the scale parameter (beta)
def exponential_log_likelihood(beta, data):
    # Ensure beta is positive to avoid errors
    beta = max(beta, 1e-6)
    # The log-likelihood for the exponential distribution is sum(log(lambda * exp(-lambda * x)))
    # where lambda = 1/beta. So log(lambda) - lambda * x
    # Summing over data: n*log(lambda) - lambda * sum(x)
    # In terms of beta: n*log(1/beta) - (1/beta) * sum(x) = -n*log(beta) - sum(x)/beta
    n = len(data)
    sum_data = np.sum(data)
    return -n * np.log(beta) - sum_data / beta

# Define reparametrization functions and their inverse functions for plotting
def reparam(param_name, original_param_values):
    if param_name == 'beta': # Original parameter
        return original_param_values
    elif param_name == 'lambda (1/beta)':
        # Avoid division by zero or very small numbers
        return 1.0 / (original_param_values + 1e-9)
    elif param_name == 'log(beta)':
        return np.log(original_param_values)

param_slider = widgets.FloatSlider(
    min=1e-1, # Use the wider range for the slider as well
    max=10,
    step=.2,
    value=1,
    description='beta=1/lambda'
)

# Create widgets
reparam_selector = widgets.Dropdown(
    options=['beta', 'lambda (1/beta)', 'log(beta)'],
    value='beta',
    description='Reparametrization:'
)

# Define a range for the original parameter (beta) for plotting
beta_values_for_plot = np.linspace(1e-1,10, 500) # Adjust range based on data

# Calculate the original log-likelihood values
original_log_likelihood_values = np.array([exponential_log_likelihood(beta, data) for beta in beta_values_for_plot])

# Function to calculate and plot log-likelihoods
def plot_reparam_likelihood(selected_reparam, param_value):
    plt.figure(figsize=(14, 6))

    # --- Original Parameterization Plot ---
    plt.subplot(1, 2, 1)
    plt.plot(beta_values_for_plot, original_log_likelihood_values, 'b-', lw=2, label='Original Log-Likelihood (beta)')
    plt.axvline(x=param_value, color='red', linestyle='--', lw=2, label=f'Selected param: {param_value:.2f}, Log-Likelihood: {exponential_log_likelihood(param_value, data):.2f}')
    plt.plot(param_value, exponential_log_likelihood(param_value, data), 'ro', markersize=8) # Add a red dot at the selected point
    plt.title('Log-Likelihood in Original Parameter Space (beta)')
    plt.xlabel('beta (scale parameter)')
    plt.ylabel('Log-Likelihood')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()

    # --- Reparameterization Plot ---
    plt.subplot(1, 2, 2)
    reparam_values_for_plot = reparam(selected_reparam, beta_values_for_plot)
    # cheat here: we plot the same y data (since de-reparametrizing will of course give the same data)
    plt.plot(reparam_values_for_plot, original_log_likelihood_values, 'r-', lw=2, label=f'Log-Likelihood (Reparametrized: {selected_reparam})')
    plt.axvline(x=reparam(selected_reparam, param_value), color='red', linestyle='--', lw=2, label=f'Selected param: {param_value:.2f}, Log-Likelihood: {exponential_log_likelihood(param_value, data):.2f}')
    plt.plot(reparam(selected_reparam, param_value), exponential_log_likelihood(param_value, data), 'ro', markersize=8) # Add a red dot at the selected point

    plt.title(f'Log-Likelihood with Reparametrization ({selected_reparam})')
    plt.xlabel(f'{selected_reparam}') # X-axis is still the original parameter
    plt.ylabel('Log-Likelihood')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()

    plt.tight_layout()
    plt.show()

# Link widgets to the plotting function
interactive_reparam_plot = widgets.interactive(
    plot_reparam_likelihood,
    selected_reparam=reparam_selector,
    param_value=param_slider,
)

# Display the widgets and the plot
display(interactive_reparam_plot)

interactive(children=(Dropdown(description='Reparametrization:', options=('beta', 'lambda (1/beta)', 'log(beta…

Vous avez désormais pu choisir différentes reparamétrisations du modèle exponentiel, et pu visualiser la fonction de log-vraisemblance correspondante.

❓En quoi cette visualisation rend-elle compte de la propriété d'invariance énoncée plus haut ?

❓Quelle propriété partagent les fonctions de reparamétrisation proposées ? Pourquoi l'invariance de la fonction de vraisemblance n'est-elle pertinente que si cette propriété est vérifiée ?

❓Les courbes de gauche et de droite sont-elles identiques ? Est-ce en désaccord avec la propriété d'invariance ?

❓Si un statisticien utilisant le premier modèle décide d'utiliser la valeur de paramètre lui donnant la plus haute vraisemblance pour les données observées, aura-t-il la même distribution que son collègue procédant de la même manière mais sur base du second modèle (celui qui est reparamétré) ?

> 🧠 Écrivez ici ce que vous en pensez !

Bien, passons désormais à la seconde propriété de la fonction de vraisemblance : l'invariance à la **transformation des données**.

Selon cette propriété, si l'on transforme les données $\mathbf{y}$ à l'aide d'une fonction injective $h(\cdot)$, obtenant ainsi $\mathbf{x}=h(\mathbf{y})$, alors la fonction de vraisemblance $L(\theta; \mathbf{x})$ est *équivalente* à la fonction de vraisemblance $L(\theta; \mathbf{y})$, où l'équivalence signifie qu'une fonction est égale à l'autre à un terme multiplicatif près.

> 🤔 Mais pour quoi faire ? Et bien, il est parfois pratique de modifier la variable considérée pour simplifier la modélisation, par exemple en passant d'une distribution log-normale à une distribution normale, ou pour respecter certaines hypothèses, par exemple en utilisant la transformation de Box-Cox.

Voyons ce que cela donne si l'on désire manipuler des données distribuées approximativement normalement, à partir de données approximativement log-normales comme celles de la variable "median_income" de notre échantillon. 👀

In [ ]:
# Load the dataframe
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

# Get the median_income data
data = df['median_income']
scale = np.std(data)

# Define the log-likelihood function for the Exponential distribution
# Using the scale parameter (beta)
def exponential_log_likelihood(beta, data):
    # Ensure beta is positive to avoid errors
    beta = max(beta, 1e-6)

    n = len(data)
    sum_data = np.sum(data)
    return -n * np.log(beta) - sum_data / beta

# Define data transformation
def data_trans(original_data):
    return np.log(original_data)

# Normal parameters
mu_min = -2.5
mu_slider = widgets.FloatSlider(
    min=mu_min, # Use the wider range for the slider as well
    max=-mu_min,
    step=.05,
    value=1,
    description='mu'
)
sigma = np.std(data_trans(data))  # fix sigma as the std of the log-transformed data

# mu values for plotting
mu_values_for_plot = np.linspace(mu_min, -mu_min, 500)

def log_normal_log_likelihood(mu, data):
    # The log-likelihood is the sum of the log of the PDFs
    return np.sum(lognorm.logpdf(data, loc=0, s=sigma, scale=np.exp(mu)))
    # return np.sum(lognorm.logpdf(data, s=np.log(scale), scale=loc))

def normal_log_likelihood(mu, data):
    # The log-likelihood is the sum of the log of the PDFs
    return np.sum(norm.logpdf(data, loc=mu, scale=sigma))

log_normal_likelihood_values = np.array([log_normal_log_likelihood(mu, data) for mu in mu_values_for_plot])
normal_likelihood_values = np.array([normal_log_likelihood(mu, data_trans(data)) for mu in mu_values_for_plot])

def plot_data_trans(mu_value):
  plt.figure(figsize=(16, 6))

  plt.subplot(2, 2, 1)
  # Plot Histogram
  plt.hist(data, bins=30, density=True, alpha=0.7, color='#1f77b4', edgecolor='black', label='Sample Histogram')
  plt.title('Sample Histogram vs. Theoretical PDF Fit')
  plt.xlabel('Median House Value')
  plt.ylabel('Density')
  plt.legend()
  plt.grid(True, linestyle='--', alpha=0.6)
  plt.xlim([data.min() - 5, data.max() + 5])
  # Plot Theoretical PDF
  x_values = np.linspace(1e-1, data.max() + 8, 500)
  plt.plot(x_values, lognorm.pdf(x_values, loc=0, s=sigma, scale=np.exp(mu_value)), 'r-', lw=2.5, label=f'Theoretical PDF (Log-normal, mu={mu_value:.2f})')
  plt.legend()

  # Plot Histogram
  plt.subplot(2, 2, 2)
  plt.hist(data_trans(data), bins=30, density=True, alpha=0.7, color='#1f77b4', edgecolor='black', label='Sample Histogram')
  plt.title('Sample Histogram vs. Theoretical PDF Fit')
  plt.xlabel('Median House Value')
  plt.ylabel('Density')
  plt.legend()
  plt.grid(True, linestyle='--', alpha=0.6)
  plt.xlim([data.min() - 5, data.max() + 5])
  # Plot Theoretical PDF
  x_values = np.linspace(data_trans(1e-1), data_trans(data.max() + 8), 500)
  plt.plot(x_values, norm.pdf(x_values, loc=mu_value, scale=sigma), 'r-', lw=2.5, label=f'Theoretical PDF (Normal, mu={mu_value:.2f})')
  plt.legend()

  # Original Data Likelihood Plot
  plt.subplot(2, 2, 3)
  plt.plot(mu_values_for_plot, log_normal_likelihood_values, 'b-', lw=2, label='Log-Likelihood')
  plt.axvline(x=mu_value, color='red', linestyle='--', lw=2, label=f'Selected param: {mu_value:.2f}, Log-Likelihood: {log_normal_log_likelihood(mu_value, data):.2f}')
  plt.plot(mu_value, log_normal_log_likelihood(mu_value, data), 'ro', markersize=8) # Add a red dot at the selected point
  plt.title('Log-Likelihood with Original Data')
  plt.xlabel(r'$\mu$')
  plt.ylabel('Log-Likelihood')
  plt.grid(True, linestyle='--', alpha=0.6)
  plt.legend()

  # Data transformation Likelihood Plot
  plt.subplot(2, 2, 4)
  plt.plot(mu_values_for_plot, normal_likelihood_values, 'r-', lw=2, label=f'Log-Likelihood')
  plt.axvline(x=mu_value, color='red', linestyle='--', lw=2, label=f'Selected param: {mu_value:.2f}, Log-Likelihood: {normal_log_likelihood(mu_value, data_trans(data)):.2f}')
  plt.plot(mu_value, normal_log_likelihood(mu_value, data_trans(data)), 'ro', markersize=8) # Add a red dot at the selected point

  plt.title(f'Log-Likelihood with Data transformation')
  plt.xlabel(r'$\mu$') # X-axis is still the original parameter
  plt.ylabel('Log-Likelihood')
  plt.grid(True, linestyle='--', alpha=0.6)
  plt.legend()

  plt.tight_layout()
  plt.show()

# Link widgets to the plotting function
interactive_reparam_plot = widgets.interactive(
    plot_data_trans,
    mu_value=mu_slider,
)

# Display the widgets and the plot
display(interactive_reparam_plot)

interactive(children=(FloatSlider(value=1.0, description='mu', max=2.5, min=-2.5, step=0.05), Output()), _dom_…

❓La variable transformée est-elle distribuée de la même façon que la variable de départ ?

❓Les deux fonctions de vraisemblance sont-elles rigoureusement identiques ? En particulier, les valeurs des fonctions pour une valeur de $\mu$ donnée sont-elles égales ? Si non, à quel facteur près ?

❓Votre réponse au point précédent remet-elle en cause la propriété énoncée précédemment ? Quelle implication cela a-t-il sur la détermination du paramètre donnant la vraisemblance la plus élevée ?

> 🧠 Écrivez ce que vous en pensez !

## Conclusion et pistes de réflexion 💭

Vous êtes arrivés à la fin de ce second notebook, félicitations. 🎉🥳

Nous y avons exploré deux concepts fondamentaux en statistiques : les modèles et la vraisemblance. Ces deux outils permettent d'aller un pas plus loin dans l'analyse de données, et touchent à la compréhension du processus générateur des données obtenues, et non plus juste à la description de celles-ci. Bien que ce travail nous apporte déjà des informations précieuses, il ne nous permet pas encore de déterminer les paramètres les plus adéquats pour décrire les données. 🤔

Dans un prochain notebook, nous allons donc nous baser sur ces fondations afin d'explorer les méthodes pour estimer les paramètres qui expliquent le mieux nos observations ! 🚀